# Импорт библиотек

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import requests
from datetime import datetime
import time
import pytz


# Составление датасета

## Константы

In [26]:
POLLUTANTS = ['pm25', 'pm10', 'so2', 'no2', 'co', 'no']
DATE_FROM = datetime(2020, 1, 1)
DATE_TO = datetime(2023, 1, 1)
url_measurments = "https://api.openaq.org/v2/measurements"
url_cities = "https://api.openaq.org/v2/cities"
url_locations = "https://api.openaq.org/v2/locations" 
url_countries = "https://api.openaq.org/v3/countries"
date_range = pd.date_range(start=DATE_FROM, end=DATE_TO, freq='ME')

with open('apikey', "r") as keyfile:
    HEADERS = {}
    HEADERS['x-api-key'] = keyfile.readline().strip()

## Получим список всех стран через api предложенного сайта, которые соответствуют необходимым фильтрам:
Минимальная и максимальная даты измерений находятся в диапазоне 3х лет между 1 января 2020 и 1 января 2023 </br>
Есть много загрязнителей (все 6, которые я указывал в константах)

In [4]:
all_countries = []

params = {
    'limit': 200,
}

response = requests.get(url_countries, headers=HEADERS, params=params)

if response.status_code == 429:
    print("Превышено количество запросов. Ожидание 5 секунд...")
    time.sleep(5)

if response.status_code != 200:
    print(f"Ошибка: {response.status_code}, {response.text}")

data = response.json()
results = data.get('results', [])

if not results:
    print(f"Нет данных для {city}")

def parse_datetime(date_string):
    try:
        return datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        return datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%SZ')


for elem in results:
    
    first = parse_datetime(elem['datetimeFirst'])
    last = parse_datetime(elem['datetimeLast'])
    parameters = [param['name'] for param in elem["parameters"]]

    if first <= DATE_FROM and last >= DATE_TO and \
        all(pollutant in parameters for pollutant in POLLUTANTS):
        all_countries.append((elem['name'], elem['id']))
print(all_countries)

[('Argentina', 6), ('Israel', 11), ('Palestine', 12), ('France', 22), ('South Africa', 37), ('Brazil', 45), ('Czech Republic', 49), ('Germany', 50), ('Sweden', 54), ('Luxembourg', 58), ('North Macedonia', 62), ('Spain', 67), ('Denmark', 71), ('Slovakia', 76), ('Poland', 77), ('United Kingdom', 79), ('Austria', 89), ('Italy', 91), ('Switzerland', 92), ('Netherlands', 94), ('Croatia', 103), ('Andorra', 129), ('United States', 155), ('Canada', 156), ('Mexico', 157), ('Malta', 223)]


## Получаем все локации, которые соответствуют тем же требованиям что и страны по 100 локаций для страны

In [17]:
all_locations = []

for country in all_countries:

    params = {
        'country_id': country[1],
        'limit': 100,
    }
    
    print(f"Запрос локаций для страны: {country}")
    
    response = requests.get(url_locations, headers=HEADERS, params=params)
    
    if response.status_code == 429:
        print("Превышено количество запросов. Ожидание 5 секунд...")
        time.sleep(5)
        continue

    if response.status_code != 200:
        print(f"Ошибка: {response.status_code}, {response.text}")
        break
    
    data = response.json()
    results = data.get('results', [])
    print(results) 
    for elem in results:
        if elem['firstUpdated'] is not None and elem['lastUpdated'] is not None:
            first = datetime.fromisoformat(elem['firstUpdated'])
            last = datetime.fromisoformat(elem['lastUpdated'])
            parameters = [param['parameter'] for param in elem["parameters"]]
            
            DATE_FROM = DATE_FROM.replace(tzinfo=pytz.UTC)
            DATE_TO = DATE_TO.replace(tzinfo=pytz.UTC)
            
            if first <= DATE_FROM and last >= DATE_TO and \
                all(pollutant in parameters for pollutant in POLLUTANTS):
                all_locations.append((elem['name'], elem['id']))
    
print(all_locations)

Запрос локаций для страны: ('Argentina', 6)
[{'id': 5240, 'city': 'Buenos Aires', 'name': 'LA BOCA', 'entity': None, 'country': 'AR', 'sources': None, 'isMobile': False, 'isAnalysis': None, 'parameters': [{'id': 8, 'unit': 'ppm', 'count': 24447, 'average': 0.29317637910924804, 'parameter': 'co', 'lastValue': 0.67, 'displayName': 'co ppm', 'lastUpdated': '2024-10-03T03:00:00+00:00', 'parameterId': 8, 'firstUpdated': '2017-08-10T23:00:00+00:00', 'manufacturers': None}, {'id': 7, 'unit': 'ppm', 'count': 24471, 'average': 0.017595304880680222, 'parameter': 'no2', 'lastValue': 0.034, 'displayName': 'no2 ppm', 'lastUpdated': '2024-10-03T03:00:00+00:00', 'parameterId': 7, 'firstUpdated': '2017-08-10T23:00:00+00:00', 'manufacturers': None}, {'id': 1, 'unit': 'µg/m³', 'count': 24458, 'average': 26.477806563039724, 'parameter': 'pm10', 'lastValue': 0.0, 'displayName': 'pm10 µg/m³', 'lastUpdated': '2024-10-03T03:00:00+00:00', 'parameterId': 1, 'firstUpdated': '2017-08-10T23:00:00+00:00', 'manufac

In [27]:
all_results = []
 

for location in all_locations:
    for month_end in date_range:
        start_date = month_end.replace(day=1)
        end_date = month_end

        params = {
            'parameter': POLLUTANTS,
            'date_from': start_date.strftime('%Y-%m-%dT00:00:00Z'),
            'date_to': end_date.strftime('%Y-%m-%dT23:59:59Z'),
            'page' : 1,
            'limit': 100,
            'location' : location[0],
            'location_id' : location[1]
        }

        print(f"Параметры запроса: {params}")
        
        while True:
            response = requests.get(url_measurments, headers=HEADERS, params=params)
            
            if response.status_code == 429:
                print("Превышено количество запросов. Ожидание 5 секунд...")
                time.sleep(5)
                continue
            
            if response.status_code != 200:
                print(f"Ошибка: {response.status_code}, {response.text}")
                break

            data = response.json()
            results = data.get('results', [])
            
            if not results:
                print(f"Нет данных для {location}, {pollutant} в период с {start_date} по {end_date}")
                break

            all_results.extend(results)

            if 'meta' in data and 'next' in data['meta']:
                params['page'] = data['meta']['next']
            else:
                break
            
            time.sleep(2)
            
df = pd.DataFrame(all_results)
df

Параметры запроса: {'parameter': ['pm25', 'pm10', 'so2', 'no2', 'co', 'no'], 'date_from': '2020-01-01T00:00:00Z', 'date_to': '2020-01-31T23:59:59Z', 'page': 1, 'limit': 100, 'location': 'Praha 4-Libus', 'location_id': 4369}
Параметры запроса: {'parameter': ['pm25', 'pm10', 'so2', 'no2', 'co', 'no'], 'date_from': '2020-02-01T00:00:00Z', 'date_to': '2020-02-29T23:59:59Z', 'page': 1, 'limit': 100, 'location': 'Praha 4-Libus', 'location_id': 4369}
Параметры запроса: {'parameter': ['pm25', 'pm10', 'so2', 'no2', 'co', 'no'], 'date_from': '2020-03-01T00:00:00Z', 'date_to': '2020-03-31T23:59:59Z', 'page': 1, 'limit': 100, 'location': 'Praha 4-Libus', 'location_id': 4369}
Параметры запроса: {'parameter': ['pm25', 'pm10', 'so2', 'no2', 'co', 'no'], 'date_from': '2020-04-01T00:00:00Z', 'date_to': '2020-04-30T23:59:59Z', 'page': 1, 'limit': 100, 'location': 'Praha 4-Libus', 'location_id': 4369}
Параметры запроса: {'parameter': ['pm25', 'pm10', 'so2', 'no2', 'co', 'no'], 'date_from': '2020-05-01T00

SSLError: HTTPSConnectionPool(host='api.openaq.org', port=443): Max retries exceeded with url: /v2/measurements?parameter=pm25&parameter=pm10&parameter=so2&parameter=no2&parameter=co&parameter=no&date_from=2021-08-01T00%3A00%3A00Z&date_to=2021-08-31T23%3A59%3A59Z&page=1&limit=100&location=Bremerhaven-Hansastra%C3%9Fe&location_id=4713 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

In [29]:
len(all_results)

19900